<a href="https://colab.research.google.com/github/hrithikwel8/Rossmann_Sales_Prediction/blob/main/Ameen_Rossmann_Sales_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Sales Prediction : Predicting sales of a major store chain Rossmann</u></b>

## <b> Problem Description </b>

### Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

### You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

## <b> Data Description </b>

### <b>Rossmann Stores Data.csv </b> - historical data including Sales
### <b>store.csv </b> - supplemental information about the stores


### <b><u>Data fields</u></b>
### Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* #### Id - an Id that represents a (Store, Date) duple within the test set
* #### Store - a unique Id for each store
* #### Sales - the turnover for any given day (this is what you are predicting)
* #### Customers - the number of customers on a given day
* #### Open - an indicator for whether the store was open: 0 = closed, 1 = open
* #### StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* #### SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* #### StoreType - differentiates between 4 different store models: a, b, c, d
* #### Assortment - describes an assortment level: a = basic, b = extra, c = extended
* #### CompetitionDistance - distance in meters to the nearest competitor store
* #### CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* #### Promo - indicates whether a store is running a promo on that day
* #### Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* #### Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* #### PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from datetime import date, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
store_data_df = pd.read_csv('/content/drive/MyDrive/AlmaBetter Data Science/CAPSTONE PROJECT-2/Retail Sales Prediction/Data & Resources/Rossmann Stores Data.csv')
store_df = pd.read_csv('/content/drive/MyDrive/AlmaBetter Data Science/CAPSTONE PROJECT-2/Retail Sales Prediction/Data & Resources/store.csv')

In [4]:
store_data_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


**Model Evaluation**

In [ ]:
# Function to check for Evaluation Metrics
def mean_absolute_percentage_error(actual, pred):
  error_term = actual - pred
  num_of_observation = len(error_term)
  sum = 0
  for i in range(num_of_observation):
    if actual[i] != 0:
      sum = sum + abs(error_term[i]/actual[i])
    else:
      sum = sum + abs(error_term[i])
  return sum*100/num_of_observation

def evaluation_metrics(actual, pred, num_of_features=0):
  MSE = mean_squared_error (actual , pred)
  print('Mean Squared Error :' , round(MSE,2))
  print('Root Mean Squared Error :' , round(np.sqrt(MSE)))
  error_term = actual - pred
  num_of_observation = len(error_term)
  MAE = sum(abs(error_term))/num_of_observation
  print('Mean Absolute Error :' , round(MAE,2))
  print('Mean Absolute Percentage Error :' , round(mean_absolute_percentage_error(actual, pred),2), '%')
  R2 = r2_score(y_test , y_pred)
  print('R-Square :' , round(R2,2)) 
  if num_of_features > 0:
    adj_r2 = 1-(1-R2)*((num_of_observation-1)/(num_of_observation-num_of_features-1))
    print("Adjusted R-Square : ", round(adj_r2, 2))

In [ ]:
# Evaluating Linear Reegression model
evaluation_metrics(y_test , y_pred, len(independent_variables))

In [ ]:
#Plot between actual and predicted
plt.figure(figsize=(8,7))
plt.plot(y_test)
plt.plot(y_pred)
plt.legend(["Predicted" , "Actual"])
plt.show()

**Checking the assumptions of Linear Regression**

In [ ]:
# Plotting the relationship between sales and customer
sns.scatterplot(data=sales_df,x='Customers_log_t',y='Sales_log_t')

In [ ]:
# Plotting the relationship between Sales and Competition Distance
sns.scatterplot(data=sales_df,x='CompetitionDistance',y='Sales_log_t')

In [ ]:
# Plotting the relationship between Sales and Compition Open Since Number Of Days
sns.scatterplot(data=sales_df,x='CompitionTime',y='Sales_log_t')

In [ ]:
# Checking the mean of Residuals
sum(y_pred-y_test)/ len(sales_df)

In [ ]:
# Checking Homoscedasticity
error_terms = y_pred-y_test
plot = sns.scatterplot(x=y_test,y=error_terms)
# plot.map_dataframe(plot_hline,y=0)
plt.plot([0,max(y_test)], [0,0], 'r-')

**Regularization**

**Implementing Lasso Regression**

In [ ]:
lasso = Lasso(alpha = 0.1 , max_iter = 3000)
lasso.fit(X_train , y_train)

In [ ]:
lasso.score(X_train , y_train)

In [ ]:
y_pred_lasso = lasso.predict(X_test)

In [ ]:
#Evaluation metrics for Lasso Regression
evaluation_metrics(y_test , y_pred_lasso, len(independent_variables))

**Running Grid Search Cross Validation for Lasso Regression**

In [ ]:
#cross validation
lasso = Lasso()
parameters = {'alpha': [1e-10,1e-8,1e-5,1e-3,1e-1,1]}
lasso_regressor = GridSearchCV(lasso , parameters , scoring='neg_mean_squared_error' , cv=5)
lasso_regressor.fit(X_train, y_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,lasso_regressor.best_params_)
print("\nUsing ",lasso_regressor.best_params_, " the negative mean squared error is: ", lasso_regressor.best_score_)

In [ ]:
y_pred_lasso = lasso_regressor.predict(X_test)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(y_pred_lasso)
plt.plot(np.array(y_test))
plt.legend(["Predicted","Actual"])
plt.show()

In [ ]:
#Evaluation metrics for Lasso Regression
evaluation_metrics(y_test , y_pred_lasso, X_test.shape[1])